# Importing the Library

In [6]:
import os
import json
import pandas as pd
import traceback

In [13]:
from langchain.chat_models import ChatGooglePalm

In [24]:
from dotenv import load_dotenv
load_dotenv() #takes environment variable from .env

True

In [28]:
KEY = os.getenv("GOOGLE_PALM_API_KEY")

In [29]:
llm = ChatGooglePalm(
    google_api_key = KEY,
    model="models/chat-bison-001",
    temperature=0.5
    )